In [ ]:
### data opening
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
train_dataset = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/train.csv')

In [ ]:
train_dataset

In [ ]:
##common info about target
train_dataset.describe()

In [ ]:
train_dataset.info()

In [ ]:
train_dataset.hist()

In [ ]:
train_dataset['context'].value_counts().plot()


In [ ]:
train_dataset['target'].value_counts().plot()
plt.tight_layout()

In [ ]:
sns.barplot(train_dataset['score'][:50:], train_dataset['target'][:50:])

In [ ]:
sns.barplot(train_dataset['score'][:50:], train_dataset['context'][:50:])

In [ ]:
import tensorflow as tf
import keras.backend as K
from keras.preprocessing.text import text_to_word_sequence, Tokenizer, one_hot
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, LSTM, Embedding, Flatten, GlobalAveragePooling1D
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
import keras

In [ ]:
import transformers
from transformers import AutoTokenizer
tokenizer= AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
#tokenizing data columns with BERTa-autotokenizer model + TFRobertaModel
## building model with embedding type nn's

In [ ]:
import transformers 
from transformers import TFRobertaModel
target=train_dataset['score']

In [ ]:
data=tokenizer([(k[0]+';'+k[1], k[2]) for k in zip(train_dataset.anchor, train_dataset.context, train_dataset.target)], padding='max_length', max_length=510)

In [ ]:
input_ids=[]
token_type_ids=[]
attention_mask=[]
for i in range(len(train_dataset.anchor)):
    input_ids.append(dict(data)['input_ids'][i])
    token_type_ids.append(dict(data)['token_type_ids'][i])
    attention_mask.append(dict(data)['attention_mask'][i])

In [ ]:
input__ids=tf.convert_to_tensor(np.array(input_ids))
token_type__ids=tf.convert_to_tensor(np.array(token_type_ids))
attention__mask=tf.convert_to_tensor(np.array(attention_mask))

In [ ]:
from transformers import TFRobertaModel
berta=TFRobertaModel.from_pretrained('bert-base-uncased', trainable=True)

In [ ]:
def create_model(input__ids, attention__mask, berta):
    input_id=tf.keras.Input(shape=(510,), dtype=tf.int32)
    input_mask=tf.keras.Input(shape=(510,), dtype=tf.int32)
    x=berta(input_ids=input_id, attention_mask=input_mask)
    x=keras.layers.GlobalAveragePooling1D()(x.last_hidden_state)
    x=BatchNormalization()(x)
    x=keras.layers.Dropout(0.3)(x)
    res=keras.layers.Dense(units=1)(x)
    model=keras.Model(inputs=[input_id, input_mask], outputs=res)
    model.compile(loss='mae', optimizer='rmsprop', metrics='mae')
    return model

In [ ]:
model=create_model(input__ids, attention__mask, berta)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
model.fit((input__ids[:10:], attention__mask[:10:]), target[:10:], epochs=3,
         verbose=1, batch_size=500, validation_data=(((input__ids[15:20:], attention__mask[15:20:])), target[15:20:]))

In [ ]:
model.predict((input__ids[:1:], attention__mask[:1:])), target[:1:]